# Levantamiento Tablas Acciones Comerciales - Código

> **PROYECTO : LEVANTAMIENTO CÓDIGO EN EL GESTOR DE CAMPANAS TERADATA** <br> 
**Extracción de tablas utilizadas en las Vistas, Bteq, SP e Input Analítico** <br>
Los archivos a procesar están en la carpeta ./ArchivosProcesar  <br>
Versión:  1.0  <br>
Fecha: 01-05-2020  <br>
Descripción: Versión Inicial  <br>
Desarrollador: Axity | Adriana Jiménez 

```
* El siguiente desarrollo está basado en la librería de python sql_metadata
* Modificando la función 'get_query_tables' según las necesidades específicas de este requerimiento

URLs:
https://pypi.org/project/sql_metadata/
https://github.com/macbre/sql-metadata/blob/master/sql_metadata.py
```

## Instalación de Librerías

In [29]:
"""
pip install pandas
pip install sql_metadata
pip install sqlparse
"""

'\npip install pandas\npip install sql_metadata\npip install sqlparse\n'

## Librerías

In [30]:
import re
import os, glob
import pandas as pd 
from decimal import Decimal as D
import datetime

## Librerias de parseo de queries
import sqlparse
from sqlparse.sql import TokenList
from sqlparse.tokens import Name, Whitespace, Wildcard, Number, Punctuation, Text, Operator
from sqlparse.tokens import DML, DDL, Keyword
import sql_metadata as sqllib

## Libreria que se integra con Teradata
import giraffez

## Variables Globales

In [31]:
noConditionInput = 'Parametros: No tiene condicion de tabla input'

## Funciones: Insertar en BD Teradata

In [32]:
## Configuracion de conexion
td_config = {
    "username": "exajibl",
    "password": "acjb0610",
    "host": "dataware.bci.cl"}

In [33]:
def create_and_drop_tables_teradata():
    '''
    Función que crea las tablas en Teradata
    '''
    
    print("ELIMINANDO Y CREANDO TABLAS..")
    drop_sql_bteq    = "DROP TABLE EDW_TEMPUSU.LAC_LEVANTAMIENTO_TABLAS_BTEQ"
    drop_sql_sp      = "DROP TABLE EDW_TEMPUSU.LAC_LEVANTAMIENTO_TABLAS_SP"
    drop_sql_vistas  = "DROP TABLE EDW_TEMPUSU.LAC_LEVANTAMIENTO_TABLAS_VISTAS"
    drop_sql_resumen = "DROP TABLE EDW_TEMPUSU.LAC_LEVANTAMIENTO_TABLAS_RESUMEN"

    create_sql_bteq = """CREATE MULTISET TABLE EDW_TEMPUSU.LAC_LEVANTAMIENTO_TABLAS_BTEQ
        (
          TIPO_ARCHIVO VARCHAR(20) ,
          ARCHIVO VARCHAR(200),
          NUMERO_PASO VARCHAR(5),      
          SENTENCIA_DML VARCHAR(100) ,
          ESQUEMA_OUTPUT VARCHAR(50) ,
          TABLA_OUTPUT VARCHAR(100) ,
          ESQUEMA_INPUT VARCHAR(50) ,
          TABLA_INPUT VARCHAR(100)      
          ) ;"""

    create_sql_sp = """CREATE MULTISET TABLE EDW_TEMPUSU.LAC_LEVANTAMIENTO_TABLAS_SP
        (
          TIPO_ARCHIVO VARCHAR(20) ,
          ARCHIVO VARCHAR(200),
          NUMERO_PASO VARCHAR(3),      
          SENTENCIA_DML VARCHAR(15) ,
          ESQUEMA_OUTPUT VARCHAR(30) ,
          TABLA_OUTPUT VARCHAR(30) ,
          ESQUEMA_INPUT VARCHAR(50) ,
          TABLA_INPUT VARCHAR(50)      
          ) ;"""

    create_sql_vistas = """CREATE MULTISET TABLE EDW_TEMPUSU.LAC_LEVANTAMIENTO_TABLAS_VISTAS
        (
          TIPO_ARCHIVO VARCHAR(20) ,
          ARCHIVO VARCHAR(200),     
          SENTENCIA_DML VARCHAR(15) ,
          ESQUEMA_OUTPUT VARCHAR(30) ,
          TABLA_OUTPUT VARCHAR(30) ,
          ESQUEMA_INPUT VARCHAR(50) ,
          TABLA_INPUT VARCHAR(50)      
          ) ;"""

    create_sql_resumen = """CREATE MULTISET TABLE EDW_TEMPUSU.LAC_LEVANTAMIENTO_TABLAS_RESUMEN
        (
          TIPO_ARCHIVO VARCHAR(20),
          ARCHIVO VARCHAR(200),
          TIPO_DE_TABLA VARCHAR(10),
          ESQUEMA VARCHAR(30),
          TABLA VARCHAR(50)
          ) ;"""

    with giraffez.Cmd(**td_config) as cmd:
        if cmd.exists("EDW_TEMPUSU.LAC_LEVANTAMIENTO_TABLAS_BTEQ"):
            cmd.execute(drop_sql_bteq)
        if cmd.exists("EDW_TEMPUSU.LAC_LEVANTAMIENTO_TABLAS_SP"):
            cmd.execute(drop_sql_sp)
        if cmd.exists("EDW_TEMPUSU.LAC_LEVANTAMIENTO_TABLAS_VISTAS"):
            cmd.execute(drop_sql_vistas)
        if cmd.exists("EDW_TEMPUSU.LAC_LEVANTAMIENTO_TABLAS_RESUMEN"):
            cmd.execute(drop_sql_resumen)


        cmd.execute(create_sql_bteq)
        cmd.execute(create_sql_sp)
        cmd.execute(create_sql_vistas)
        cmd.execute(create_sql_resumen)

In [34]:
def insert_files_csv_in_teradata():
    '''
    Función que inserta los dataframes a Teradata
    '''
    
    print("INSERTANDO ARCHIVOS A LAS TABLAS..\n")

    
    df_bteq    = pd.read_csv("LAC_Levantamiento_Tablas_Bteq.csv",  sep='|' ) 
    df_sp      = pd.read_csv("LAC_Levantamiento_Tablas_StoredProcedures.csv" ,  sep='|') 
    df_vistas  = pd.read_csv("LAC_Levantamiento_Tablas_Vistas.csv",  sep='|') 
    #df_resumen = pd.read_csv("LAC_Levantamiento_Tablas_Resumen.csv",  sep='|' ) 

    
    if (df_bteq.empty == False): 
        
        print("INSERTANDO EN EDW_TEMPUSU.LAC_LEVANTAMIENTO_TABLAS_BTEQ")
        
        df_bteq['NUMERO_PASO'] = df_bteq['NUMERO_PASO'].astype('str')
        
        with giraffez.BulkLoad("EDW_TEMPUSU.LAC_LEVANTAMIENTO_TABLAS_BTEQ", **td_config) as load:
            load.cleanup()
            load.columns = df_bteq.columns.tolist()
            for row in df_bteq.values.tolist(): 
                load.put([row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7]])

    
    if (df_sp.empty == False):
        
        print("INSERTANDO EN EDW_TEMPUSU.LAC_LEVANTAMIENTO_TABLAS_SP")
        
        df_sp['TIPO_ARCHIVO']   = df_sp['TIPO_ARCHIVO'].astype('str')
        df_sp['ARCHIVO']        = df_sp['ARCHIVO'].astype('str')
        df_sp['NUMERO_PASO']    = df_sp['NUMERO_PASO'].astype('str')
        df_sp['SENTENCIA_DML']  = df_sp['SENTENCIA_DML'].astype('str')
        df_sp['ESQUEMA_OUTPUT'] = df_sp['ESQUEMA_OUTPUT'].astype('str')
        df_sp['TABLA_OUTPUT']   = df_sp['TABLA_OUTPUT'].astype('str')
        df_sp['ESQUEMA_INPUT']  = df_sp['ESQUEMA_INPUT'].astype('str')
        df_sp['TABLA_INPUT']    = df_sp['TABLA_INPUT'].astype('str')
        
        
        with giraffez.BulkLoad("EDW_TEMPUSU.LAC_LEVANTAMIENTO_TABLAS_SP", **td_config) as load:
            load.cleanup()
            load.columns = df_sp.columns.tolist()
            for row in df_sp.values.tolist(): 
                load.put([row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7]])  

    if (df_vistas.empty == False): 
        
        print("INSERTANDO EN EDW_TEMPUSU.LAC_LEVANTAMIENTO_TABLAS_VISTAS")
        
        
        df_vistas['TIPO_ARCHIVO']   = df_vistas['TIPO_ARCHIVO'].astype('str')
        df_vistas['ARCHIVO']        = df_vistas['ARCHIVO'].astype('str')
        df_vistas['SENTENCIA_DML']  = df_vistas['SENTENCIA_DML'].astype('str')
        df_vistas['ESQUEMA_OUTPUT'] = df_vistas['ESQUEMA_OUTPUT'].astype('str')
        df_vistas['TABLA_OUTPUT']   = df_vistas['TABLA_OUTPUT'].astype('str')
        df_vistas['ESQUEMA_INPUT']  = df_vistas['ESQUEMA_INPUT'].astype('str')
        df_vistas['TABLA_INPUT']    = df_vistas['TABLA_INPUT'].astype('str')
 
            
        with giraffez.BulkLoad("EDW_TEMPUSU.LAC_LEVANTAMIENTO_TABLAS_VISTAS", **td_config) as load:
            load.cleanup()
            load.columns = df_vistas.columns.tolist()
            for row in df_vistas.values.tolist(): 
                load.put([row[0], row[1], row[2], row[3], row[4], row[5], row[6] ]) 

    '''
    if (df_resumen.empty == False): 
        with giraffez.BulkLoad("EDW_TEMPUSU.LAC_LEVANTAMIENTO_TABLAS_RESUMEN", **td_config) as load:
            load.cleanup()
            load.columns = df_resumen.columns.tolist()
            for row in df_resumen.values.tolist(): 
                load.put([row[0], row[1], row[2], row[3], row[4]]) 
    '''

## Funciones modificadas de librería sql_metadata

In [35]:
def unique(_list):
    """
    Hace que una lista tenga registro unicos y mantengan el orden
    """
    ret = []

    for item in _list:
        if item not in ret:
            ret.append(item)

    return ret

In [36]:
def get_query_tables(file_type, filename, query):
    """
    Función que retorna una lista de tablas INPUT de una query
    """
    
    tables              = []
    tables_2            = []
    table_list          = []
    table_list_total    = []
    last_keyword  = None
    last_token    = None
    dml_ddl_name  = None
    not_get_from  = False
    from_clausule = False


    df_internal_table   = emptyDataframeTablesQuery(file_type)
    
    #     
    dates_list = ['SECOND', 'MINUTE', 'HOUR', 'DAY', 'YEAR','MONTH','BOTH', 'TRAILING']
    
    stop_from_list = ['AS','CASE','WHEN','ON','AND','END','WHERE','GROUP','OVER','PARTITION','SET']

    functions_ignored = ['COUNT', 'MIN', 'MAX', 'SUM', 'FROM_UNIXTIME', 'DEC', 
    'CAST', 'CONVERT', 'ZEROIFNULL','SUBSTR','SUBSTRING','ROW_NUMBER','QUALIFY', 'ADD_MONTHS',
    'COALESCE', 'CHAR', 'INTEGER', 'TRIM', 'OVER', 'FORMAT', 'DATE_FORMAT',
    'CHAR_LENGTH']

    # Lista de funciones reservadas de SQL ANSI para las uniones de tablas
    table_syntax_joins = ['FROM','JOIN', 'INNER JOIN', 'LEFT JOIN', 'LEFT OUTER JOIN', 
    'RIGHT JOIN', 'RIGHT OUTER JOIN']

    table_syntax_keywords = [
        # SELECT queries
        'FROM', 'WHERE', 'JOIN', 'INNER JOIN', 'LEFT JOIN', 'LEFT OUTER JOIN', 
        'RIGHT JOIN', 'RIGHT OUTER JOIN', 'ON',
        
        # INSERT queries
        'INTO', 'VALUES',
        # UPDATE queries
        'UPDATE', 'SET', 
        # Hive queries
        'TABLE',  # INSERT TABLE
    ]
    
    archivo_caido = 'SQLAExport1827.txt'
    
    for token in sqllib.get_query_tokens(query):
        
        token_value_clean = " ".join(token.value.upper().split())
        
        ## Homologar en caso que se consiga un 'SEL' en vez de 'SELECT'
        if ((token.ttype is Name and token.value.upper() == 'SEL')):
            token.ttype = DML
            token.value = 'SELECT'
            last_keyword = 'SELECT'
            
        if (token.ttype is DML or token.ttype is DDL):
            dml_ddl_name = token.value.upper()  
            from_clausule = False      
        
        if token.ttype is Punctuation and token.value == ',' \
            and from_clausule == True:
            last_keyword = 'FROM'
        
        if token.is_keyword and from_clausule == True and \
           token.value.upper() in stop_from_list:            
            from_clausule = False
            last_keyword = None
            
            
        
        ## OBTIENE TABLAS
        '''
        Sección que recupera las tablas de una query 
        Retorna una lista con el formato: [esquema, tabla, alias]
        '''          
        if token.is_keyword and token_value_clean in table_syntax_keywords:
            # keep the name of the last keyword, the next one can be a table name
            last_keyword = " ".join(token.value.upper().split())
            
            ## Para los casos de algunos campos en el SELECT que llaman a FROM
            if token.value.upper() == 'FROM' and last_token in dates_list:
                not_get_from = True
                
            elif(token.value.upper() == 'FROM' and not_get_from == True):
                not_get_from = False
                
            ## Para los casos de que las tablas estén separadas por ,    
            elif(token.value.upper() == 'FROM' and last_token in ["'"+'0'+"'"]):
                from_clausule = False
                not_get_from  = True

            elif(token.value.upper() == 'FROM' and last_token not in["'"+'0'+"'"]):
                from_clausule = True

                
            elif(token.value.upper() in ['WHERE','GROUP']):
                from_clausule = False
            
        elif str(token) == '(' or str(token) == ')':
            #print(" ENTRO 2 " )
            # reset the last_keyword for INSERT `foo` VALUES(id, bar) ...
            last_keyword = None
        elif token.is_keyword and str(token) in ['FORCE', 'ORDER']:
            #print(" ENTRO 3 " )
            # reset the last_keyword for "SELECT x FORCE INDEX" queries and "SELECT x ORDER BY"
            last_keyword = None
        elif token.is_keyword and str(token) == 'SELECT' and last_keyword in ['INTO', 'TABLE']:
            # reset the last_keyword for "INSERT INTO SELECT" and "INSERT TABLE SELECT" queries
            last_keyword = None
            #print(" ENTRO 4 " )
        elif token.ttype is Name or token.is_keyword:
            # print([last_keyword, last_token, token.value])
            # analyze the name tokens, column names and where condition values

            if token.ttype is Name and token.value in functions_ignored \
               and from_clausule == True :
                not_get_from = True


            if last_keyword in ['FROM', 'JOIN', 'INNER JOIN', 'LEFT JOIN', 'RIGHT JOIN',
                                'LEFT OUTER JOIN','RIGHT OUTER JOIN', 'UPDATE' ] \
                    and last_token not in ['AS', 'CREATE'] \
                    and token.value not in ['AS', 'SELECT'] \
                    and not_get_from == False:

                if last_token == '.':
                    
                    # we have database.table notation example
                    # append table name to the last entry of tables
                    # as it is a database name in fact
                    database_name = tables[-1]
                    table = token.value
                    tables[-1] = '{}.{}'.format(database_name, token)
                    last_keyword = None
                    
                    table_list = [database_name, table]
                    if (dml_ddl_name not in ['DROP']):
                        table_list_total.append(table_list)


                elif last_token not in [',', last_keyword] and \
                    last_keyword not in table_syntax_joins:
                    # it's not a list of tables, e.g. SELECT * FROM foo, bar
                    # hence, it can be the case of alias without AS, e.g. SELECT * FROM foo bar
                    pass
                else: #esquema                         
                    table_name = str(token.value.strip('`'))
                    tables.append(table_name)
                
                            
        last_token = token.value.upper()

    
    return unique(table_list_total)   

## Funciones de procesamiento de query

In [37]:
def remove_comments(text):
    """ remove c-style comments.
        text: blob of text with comments (can include newlines)
        returns: text with comments removed
    """
    pattern = r"""
                            ##  --------- COMMENT ---------
           /\*              ##  Start of /* ... */ comment
           [^*]*\*+         ##  Non-* followed by 1-or-more *'s
           (                ##
             [^/*][^*]*\*+  ##
           )*               ##  0-or-more things which don't start with /
                            ##    but do end with '*'
           /                ##  End of /* ... */ comment
         |                  ##  -OR-  various things which aren't comments:
           (                ## 
                            ##  ------ " ... " STRING ------
             "              ##  Start of " ... " string
             (              ##
               \\.          ##  Escaped char
             |              ##  -OR-
               [^"\\]       ##  Non "\ characters
             )*             ##
             "              ##  End of " ... " string
           |                ##  -OR-
                            ##
                            ##  ------ ' ... ' STRING ------
             '              ##  Start of ' ... ' string
             (              ##
               \\.          ##  Escaped char
             |              ##  -OR-
               [^'\\]       ##  Non '\ characters
             )*             ##
             '              ##  End of ' ... ' string
           |                ##  -OR-
                            ##
                            ##  ------ ANYTHING ELSE -------
             .              ##  Anything other char
             [^/"'\\]*      ##  Chars which doesn't start a comment, string
           )                ##    or escape
    """
    regex = re.compile(pattern, re.VERBOSE|re.MULTILINE|re.DOTALL)
    noncomments = [m.group(2) for m in regex.finditer(text) if m.group(2)]

    return "".join(noncomments)

In [38]:
def get_tables_output(query):
    ''' 
    Recorre los tokens de 'INSERT, REPLACE VIEW, CREATE' hasta conseguir un 'SELECT'
    Retorna lista: [tipo de STMT, esquema output, tabla output]
          Ejemplo: ['CREATE TABLE', 'ARMWORK','RM_BANCA_A']
    '''
    list_output = []
    create_seen         = False
    replace_seen        = False
    rename_seen         = False
    drop_seen           = False
    last_token          = None

    for token in sqllib.get_query_tokens(query):
        
        if (token.ttype is DML and token.value.upper() == 'SELECT') \
        or (token.ttype is Punctuation and token.value in ['(',';']) \
        or (token.ttype is Keyword and token.value == 'VALUES') :
            return list_output
            
        elif token.ttype is DML and \
            token.value.upper() in ['INSERT','UPDATE', 'DELETE'] :
            list_output.append(token.value.upper())
        
        elif (last_token == 'DELETE' and token.value.upper() != 'FROM' and len(token.value) <= 3):
            pass
        
        elif (last_token == 'UPDATE' and len(token.value) <= 3):
            pass
            
        elif token.ttype is DDL and token.value.upper() == 'CREATE':
            token_str = 'CREATE'
            create_seen = True

        elif token.is_keyword and token.value.upper() == 'RENAME':
            rename_seen = True            
  
        elif token.is_keyword and token.value.upper() == 'TABLE' and rename_seen:
            token_str = 'RENAME TABLE'
            list_output.append(token_str)            
            
        elif token.ttype is DDL and token.value.upper() == 'DROP':
            token_str = 'DROP'
            drop_seen = True
            list_output.append(token_str)

        elif token.is_keyword and token.value.upper() == 'JOIN' and drop_seen:
            token_str = 'DROP JOIN INDEX'
            list_output[-1] = token_str
        
        elif token.is_keyword and token.value.upper() == 'INDEX' and drop_seen:
            token_str = token_str + 'INDEX'
            list_output[-1] = token_str
            
        elif token.is_keyword and token.value.upper() == 'INDEX' and create_seen:
            list_output.append('CREATE INDEX')
 
        elif token.is_keyword and token.value.upper() == 'JOIN' and create_seen:
            token_str = 'CREATE JOIN INDEX'
            create_seen = False
            list_output.append(token_str)

        elif token.ttype is Name and token.value.upper() == 'SET' and create_seen:
            token_str = 'CREATE SET TABLE'
            create_seen = False
            list_output.append(token_str)

        elif token.ttype is Name and token.value.upper() == 'MULTISET' and create_seen:
            token_str = 'CREATE MULTISET TABLE'
            create_seen = False
            list_output.append(token_str)
            
            '''    
            elif token.ttype is Name and token.value.upper() == 'VOLATILE' and create_seen:
                token_str = 'CREATE MULTISET VOLATILE TABLE'
                create_seen = False
                list_output.append(token_str)
            '''    
            
        elif token.is_keyword and token.value.upper() == 'TABLE' and create_seen:
            token_str = 'CREATE TABLE'
            list_output.append(token_str)

        elif token.is_keyword and token.value.upper() == 'VIEW' and create_seen:
            token_str = 'CREATE VIEW'
            list_output.append(token_str)
        
        elif token.ttype is DML and token.value.upper() == 'REPLACE':
            replace_seen = True
            
        elif token.is_keyword and token.value.upper() == 'VIEW' and replace_seen:
            token_str = 'REPLACE VIEW'
            list_output.append(token_str)
        
        elif token.ttype is Name and token.value == 'HASH':
            token_str = token_str + ' HASH '
            
        elif token.ttype is Name and token.value not in ['HASH']:
            list_output.append(token.value)
            
        
        last_token = token.value.upper() 
                   
    return 'DESCONOCIDO'    
     

In [39]:
def deleteInconsistencyQuery(queryString):
    '''
    Función que elimina caracteres varios de una query
    '''    
    characters = '[%#${}]'
    finalQuery = re.sub(characters, '', str(queryString))
    return finalQuery

In [40]:
def preprocess_query(queryString):
    '''
    Función que limpia y ordena una query para ser procesada posteriormente
    '''
    queryString = queryString.replace('"','')
    queryString = sqllib.preprocess_query(queryString)
    queryString = deleteInconsistencyQuery(queryString)
    return queryString

In [41]:
def processing_queries(queryString):
    '''
    Función que procesa una query extrayendo solamente su tabla ouput
    '''
    listaQuery = []
    list_syntax_large = ['CREATE MULTISET TABLE', 'DROP JOIN INDEX']
    
    if (queryString):

        parsed                  = sqlparse.parse(queryString)[0]
        tipo_DML_DDL_principal  = parsed.token_first().value.upper()
        lista_tabla_output      = get_tables_output(queryString)
            
        if (lista_tabla_output[0] in ['DROP']):
            tabla_completa  = sqllib.get_query_tables(queryString)[0]             
            base_datos      = tabla_completa.split('.')[0]
            tabla           = tabla_completa.split('.')[1]
            
            listaQuery.append(tipo_DML_DDL_principal)
            listaQuery.append(base_datos) 
            listaQuery.append(tabla) 
            
        else:
            return lista_tabla_output

    
    return listaQuery


In [42]:
def get_input_output_tables(folder, filename, queryString):
    '''
    Función que procesa una query retornando su tabla output y la lista de tablas input
    Si no existen tablas input, se coloca un mensaje que indica que no tiene condiciones
    '''
    list_table_input  = []
    list_table_output = []

    list_syntax_large = ['CREATE MULTISET TABLE', 'DROP JOIN INDEX']
    
    if (queryString):

        parsed                  = sqlparse.parse(queryString)[0]
        type_DML_DDL_principal  = parsed.token_first().value.upper()
        list_table_output       = get_tables_output(queryString)
                
        if (type_DML_DDL_principal in ['DELETE', 'UPDATE']):
                        
            list_table_input = get_query_tables(folder, filename, queryString) 
            
            if (list_table_input):
                output_table     = list_table_input[0]

                list_table_output.clear()            
                list_table_output.append(type_DML_DDL_principal)
                list_table_output.append(output_table[0])   
                list_table_output.append(output_table[1])  

                list_table_input.remove(output_table)
                            
        elif (list_table_output[0] in ['DROP']):
            output_table    = sqllib.get_query_tables(queryString)[0] 
            l_output_table  = output_table.split('.')
            len_l_output_table = len(l_output_table)

            # Para los casos que hay tablas volatiles sin esquema
            if (len_l_output_table == 2):
                database    = l_output_table[0]
                table       = l_output_table[1]

                list_table_output.clear() 
                list_table_output.append(type_DML_DDL_principal)
                list_table_output.append(database) 
                list_table_output.append(table) 

            elif (len_l_output_table == 1):
                table       = l_output_table[0]

                list_table_output.clear() 
                list_table_output.append(type_DML_DDL_principal)
                list_table_output.append('N/A') 
                list_table_output.append(table) 
            
        else:
            list_table_input = get_query_tables(folder, filename, queryString) 

    ## Si no tiene tablas, agregar info de no tener condicion de tablas input
    if (len(list_table_input) == 0): 
        result = [ noConditionInput, noConditionInput ]
        list_table_input.append(result)
    
    return list_table_input, list_table_output

## Funciones: Insertar en Dataframes

In [43]:
def emptyDataframeTablesQuery(file_type):
    '''
    Función que retorna un objeto dataframe vacío para obtener las tablas, campos y reglas de una query    
    Formato ['TIPO_ARCHIVO','ARCHIVO','NUMERO_PASO','SENTENCIA_DML','ESQUEMA_OUTPUT','TABLA_OUTPUT',
            'ESQUEMA_INPUT','TABLA_INPUT']
    '''
    
    columnsDF = []

    if (file_type in ['Bteq','StoredProcedures']):
        columnsDF = [
            'TIPO_ARCHIVO','ARCHIVO', 
            'NUMERO_PASO','SENTENCIA_DML',
            'ESQUEMA_OUTPUT','TABLA_OUTPUT',
            'ESQUEMA_INPUT','TABLA_INPUT'
        ]

    elif (file_type == 'Vistas'):  
        columnsDF = [
            'TIPO_ARCHIVO','ARCHIVO', 
            'SENTENCIA_DML',
            'ESQUEMA_OUTPUT','TABLA_OUTPUT',
            'ESQUEMA_INPUT','TABLA_INPUT'
        ] 


    
    df = pd.DataFrame(columns = columnsDF) 
    return df

In [44]:
def emptyDataframeTablesSummary(data_list):
    '''
    Función que retorna un objeto dataframe vacío 
    para obtener las tablas (input, process y output) de un archivo procesado
    
    Formato ['TIPO_ARCHIVO','ARCHIVO','TABLA_INPUT','TABLA_PROCESS','TABLA_OUTPUT']
    '''
    
    columnsDF = ['TIPO_ARCHIVO','ARCHIVO','TIPO_DE_TABLA','ESQUEMA','TABLA']
    
    if (data_list is not None):
        df = pd.DataFrame(data_list, columns = columnsDF) 
    else:        
        df = pd.DataFrame(columns = columnsDF) 
    return df

In [45]:
def insertDataframeTablesQuery(file_type,file,process_number,output_table,input_tables_list):
    '''
    Función que inserta en un dataframe los objetos de una query
    '''

    new_df_tables = emptyDataframeTablesQuery(file_type)
        
    #print (file +"  *** LISTA TABLAS INPUT -> "  + str(input_tables_list))  
    #print (file +"  *** LISTA TABLAS OUTPUT -> "  + str(output_table)) 
 
    if (file_type in ['Bteq','StoredProcedures']):
        if (input_tables_list):
            for item in input_tables_list:
                
                if (len(output_table) == 3):
                    new_df_tables = new_df_tables.append(
                        [ {'TIPO_ARCHIVO'    : file_type, 
                            'ARCHIVO'        : file,
                            'NUMERO_PASO'    : process_number,
                            'SENTENCIA_DML'  : output_table[0],
                            'ESQUEMA_OUTPUT' : output_table[1],
                            'TABLA_OUTPUT'   : output_table[2],
                            'ESQUEMA_INPUT'  : item[0],
                            'TABLA_INPUT'    : item[1] }], 
                            ignore_index=True, sort=False) 
        
        elif (output_table[0] == 'RENAME TABLE' \
         and len(output_table) == 5):
            new_df_tables = new_df_tables.append(
                [ {'TIPO_ARCHIVO'    : file_type, 
                    'ARCHIVO'        : file,
                    'NUMERO_PASO'    : process_number,
                    'SENTENCIA_DML'  : output_table[0],
                    'ESQUEMA_OUTPUT' : output_table[3],
                    'TABLA_OUTPUT'   : output_table[4],
                    'ESQUEMA_INPUT'  : output_table[1],
                    'TABLA_INPUT'    : output_table[2] }], 
                    ignore_index=True, sort=False) 
            
        
        else:
            new_df_tables = new_df_tables.append(
                [ {'TIPO_ARCHIVO'    : file_type, 
                    'ARCHIVO'        : file,
                    'NUMERO_PASO'    : process_number,
                    'SENTENCIA_DML'  : output_table[0],
                    'ESQUEMA_OUTPUT' : output_table[1],
                    'TABLA_OUTPUT'   : output_table[2],
                    'ESQUEMA_INPUT'  : '',
                    'TABLA_INPUT'    : '' }], 
                    ignore_index=True, sort=False) 
            
    elif (file_type == 'Vistas'):
        for item in input_tables_list:
            new_df_tables = new_df_tables.append(
                [ {'TIPO_ARCHIVO'    : file_type, 
                    'ARCHIVO'        : file,
                    'SENTENCIA_DML'  : output_table[0],
                    'ESQUEMA_OUTPUT' : output_table[1],
                    'TABLA_OUTPUT'   : output_table[2],
                    'ESQUEMA_INPUT'  : item[0],
                    'TABLA_INPUT'    : item[1] }], 
                    ignore_index=True, sort=False)  
        
    return new_df_tables
   

In [46]:
def returnSummaryDataframeTables(df, file_type):
    '''
    Función que genera un resumen de que tipo de objeto es cada tabla dentro de una query
    Tipos de objetos: OUTPUT, INPUT, PROCESO
    '''

    df_summary      = emptyDataframeTablesSummary(None)
    process_list    = ['DROP']
    input_list      = ['INSERT']
    output_list     = ['RENAME VIEW','CREATE VIEW','REPLACE VIEW'] 
    columns_output  = ['TIPO_ARCHIVO','ARCHIVO','TABLA_OUTPUT','ESQUEMA_OUTPUT']
    columns_input   = ['TIPO_ARCHIVO','ARCHIVO','TABLA_INPUT','ESQUEMA_INPUT']
    df_process      = pd.DataFrame(columns = columns_output)
    df_input        = pd.DataFrame(columns = columns_input)
    df_output       = pd.DataFrame(columns = columns_output)


    if (df.empty == False): 

        if (file_type == 'Vistas'):

            df_input  = df[columns_input].groupby(["TIPO_ARCHIVO", "ARCHIVO","TABLA_INPUT","ESQUEMA_INPUT"]).\
                        last().reset_index()
                    
            df_output = df[columns_output].groupby(["TIPO_ARCHIVO", "ARCHIVO","TABLA_OUTPUT","ESQUEMA_OUTPUT"]).\
                        last().reset_index()            


        elif (file_type in ['Bteq','StoredProcedures']): ## obtener el ultimo INSERT de los pasos

            
            ## Tablas que son PROCESO: Obtiene las tablas que se destruyen.
            filter_df = df.loc[df['SENTENCIA_DML'].isin(process_list)]

            df_process = filter_df[columns_output].\
                   groupby(["TIPO_ARCHIVO", "ARCHIVO","TABLA_OUTPUT","ESQUEMA_OUTPUT"]).last().reset_index()

            df_process['TIPO_DE_TABLA'] = 'PROCESO'

            df_process = df_process.rename(columns={'TABLA_OUTPUT': 'TABLA','ESQUEMA_OUTPUT': 'ESQUEMA'})
            
            
            
            
            ## Tablas que son INPUT: Obtiene las tablas que se encuentran en un SELECT.
            filter_df = df.loc[df['SENTENCIA_DML'].isin(input_list)]

            ## para que no retorne el mensaje de cuando no tiene tablas como input
            filter   = ~filter_df['TABLA_INPUT'].str.contains(noConditionInput)

            df_input = filter_df[columns_input].where(filter).\
                         groupby(["TIPO_ARCHIVO", "ARCHIVO","TABLA_INPUT","ESQUEMA_INPUT"]).last().reset_index()
            
            
            
            
            ## Tablas que son OUTPUT: Obtiene la última tabla que fue insertada en el proceso.
            filter_df = df.loc[df['SENTENCIA_DML'].isin(input_list)]  

            filter    = ~filter_df['TABLA_OUTPUT'].str.contains('LOG') ## descartar las tablas LOG

            filter_df = filter_df.where(filter).dropna()

            df_output = filter_df[-1:]  

            df_output = df_output[columns_output]


        ## Agrega campo de tipo de tabla y modifica los nombre estandar    
        df_input['TIPO_DE_TABLA'] = 'INPUT'
        df_input  = df_input.rename(columns={'TABLA_INPUT': 'TABLA','ESQUEMA_INPUT': 'ESQUEMA'})

        df_output['TIPO_DE_TABLA'] = 'OUTPUT'
        df_output = df_output.rename(columns={'TABLA_OUTPUT': 'TABLA','ESQUEMA_OUTPUT': 'ESQUEMA'})

      
    
        ## Inserta resultados en dataframe resumen
        if (df_process.empty == False): 
            
            df_process = df_process.drop_duplicates()
            df_summary = df_summary.append(df_process,ignore_index=True,sort=False)
            
            
        if (df_input.empty == False):
            
            df_input   = df_input.drop_duplicates()
            df_summary = df_summary.append(df_input,ignore_index=True,sort=False)
                
                
        if (df_output.empty == False):
            
            df_output  = df_output.drop_duplicates()
            df_summary = df_summary.append(df_output,ignore_index=True,sort=False)

            
            
    return df_summary
   

## Funciones de procesamiento de archivos

In [47]:
def remove_files_csv():
    '''
    Función que elimina del directorio local los archivos antiguos de levantamiento
    '''
    for filename in glob.glob("./LAC_Levantamiento_Tablas_*.csv"):
        os.remove(filename) 

In [48]:
def fast_scandir_files(path):
    '''
    Función que escanea los directorios y subdirectorios,
    obteniendo como resultado una lista de archivo con,
    formato: ./Directorio/Subdirectorio/archivo
    '''

    file_list = []
    
    for root, dirs, files in os.walk(path):
        for file in files:
            if(file.endswith(".sql") or file.endswith('.txt')):
                file_list.append(os.path.join(root,file))
                
    return file_list

In [49]:
def folder_contains_name(folder_dir):
    '''
    Función que retorna que tipo de archivo se está procesando
    '''
    
    if (folder_dir.find('Bteq') != -1 or folder_dir.find('BTEQ') != -1):
        return 'Bteq'
    if (folder_dir.find('Vistas') != -1):
        return 'Vistas'
    if (folder_dir.find('StoredProcedures') != -1):
        return 'StoredProcedures'
    
    return 'Bteq'

In [50]:
def insert_file_errors(MyFile, file_type, archivo_no_procesado):
    '''
    Función que inserta en un archivo que por alguna razón da error
    '''
    MyFile.write("%s" % 'Archivo tipo *'+ str(file_type) + '*: '+archivo_no_procesado+'\n')

In [51]:
def insert_dataframe_in_csv(df, file_type):
    '''
    Función que crea un archivo de levantamiento desde un dataframe
    '''
    
    filename = 'LAC_Levantamiento_Tablas_'+file_type+'.csv'
    
    with open(filename, 'a') as f:
        df.to_csv(f, sep='|', index=None, header=f.tell()==0)


In [52]:
def insert_summary_dataframe_in_csv(df): 
    '''
    Función que crea un archivo de resumen de levantamiento desde un dataframe 
    ''' 
    
    filename = 'LAC_Levantamiento_Tablas_Resumen.csv'
    
    with open(filename, 'a') as f:
        df.to_csv(f, sep='|', index=None, header=f.tell()==0)


In [53]:
def processing_files(folder, filename):
    '''
    Función que abre un archivo y lo procesa linea por linea, eliminando sus comentarios 
    y retornando las querys a procesar en una lista
    '''

    queryVista           = ''    
    
    ## Lista de sintaxis SQL que debería tomar cada linea del archivo,
    ## contiene espacio al final porque hay campos que pueden llamarse "Update_dttm"
    ## y tomarlo como una palabra reservada
    dml_dll_syntax_query = [
        'INSERT', 'CREATE ', 
        'UPDATE ', 
        'DROP ', 'DELETE ','DELETE',
        'RENAME '
    ]
    
    if os.path.exists(filename):
        
        archivo         = open(filename, encoding="utf8",errors='ignore')
        textoArchivo    = archivo.read()
        listaQuery      = []
        sum_lines       = ''
        
        ## BTEQ Y STORED PROCEDURES 
        if (folder in ['Bteq','StoredProcedures']):
            
            bteq_text = re.sub('\-\-.*?\n|\/\*.*?\*\/', ' ', textoArchivo)  
                                    
            bteq_text = remove_comments(bteq_text)
                        
            lines = bteq_text.splitlines();
            
            for line in lines: 
                # elimina espacios a comienzo de la linea y lo edita a mayuscula
                line = line.strip().upper()                
                                    
                # comienza agregando las querys que interesan
                if line.startswith(tuple(dml_dll_syntax_query)) and not line.startswith('CREATE INDEX'): 
                    sum_lines = line
                    if line.endswith(";"):
                        listaQuery.append(line)
                        sum_lines = ''                        
                        
                # insertar en lista cuando termina la query
                elif (line.endswith(";") and sum_lines != ''): 
                    sum_lines = sum_lines + ' '+ line
                    listaQuery.append(sum_lines)  
                    sum_lines = ''      
                    
                # acumula en una variable hasta conseguir un ;
                elif (sum_lines != ''):                
                    sum_lines = sum_lines + ' '+ line
                           
        ## VIEWS
        elif (folder == 'Vistas'):
            # LIMPIA EL CODIGO SQL PARA QUE NO TENGA NINGUN TIPO DE COMENTARIO
            queryVista = re.sub('\-\-.*?\n|\/\*.*?\*\/', ' ', textoArchivo)   

            queryVista = remove_comments(queryVista)
            
            queryVista = queryVista.replace('\\', '')  
            queryVista = queryVista.replace('"', '') 
            
            listaQuery.append(queryVista)
   
    return listaQuery


In [54]:
def loop_folders_files(ppal_dir):   
    '''
    Función que itera en los directorios para obtener los archivos que se 
    quieren procesar (bteq, sp, vistas)
    '''

    MyFile=open('archivos_no_procesados.txt','w')
    
    ## Elimina los archivos de levantamiento
    remove_files_csv()
    
    folder_dir       = None      # Ruta completa de cada carpeta a leer
    result_folder    = None 
    
    # para pruebas
    filename2        = 'SQLAExport1827.txt'
    
    ## Obtiene los nombres de los archivo de extracción con sus carpetas y subcarpetas.    
    folder_list = fast_scandir_files(ppal_dir)    
    folder_list.sort()

    
    for folder_file in folder_list:
        
        ## Retorna nombre corto del tipo de archivo que se está procesando (Bteq, Vista, StoredProcedure)
        result_folder = folder_contains_name(folder_file)
        
        if result_folder is not None:
            
            print(folder_file)

            ## inserta en un dataframe el contenido por query                       
            df_internal_table      = emptyDataframeTablesQuery(result_folder)

            ## inserta en un dataframe el contenido por archivo completo                    
            df_internal_table_file = emptyDataframeTablesQuery(result_folder)

            ## insertar en un dataframe resumen las tablas por Input, Proceso, Output
            df_internal_summary    = emptyDataframeTablesSummary(None)

            listaQuerys = processing_files(result_folder, folder_file)

            if(len(listaQuerys) == 0):
                insert_file_errors(MyFile, result_folder, folder_file)

            process_number = 0

            for queryString in listaQuerys:

                process_number += 1

                if (queryString):
                    #print(queryString)
                    try:
                        queryString    = preprocess_query(queryString)


                        # Retorna todas las tablas input y output en listas
                        input_tables, output_tables = get_input_output_tables(result_folder, folder_file, queryString)

                        # Retorna todas las tablas input y output en dataframe
                        df_internal_table = insertDataframeTablesQuery(result_folder, folder_file,\
                                          process_number, output_tables, input_tables)


                        if (df_internal_table.empty == False):
                            insert_dataframe_in_csv(df_internal_table, result_folder)
                            df_internal_table_file = df_internal_table_file.append(df_internal_table)
                        else:
                            insert_file_errors(MyFile, result_folder, folder_file)   
                    
                    
                    except Exception as e: 
                        print("Error with file: "+folder_file)
                        print(e)
                        insert_file_errors(MyFile, result_folder, folder_file)
                        pass  
                      
            '''
            df_internal_summary    = returnSummaryDataframeTables(df_internal_table_file, result_folder)

            if (df_internal_summary.empty == False):
                insert_summary_dataframe_in_csv(df_internal_summary)
            '''
                          
    MyFile.close()  

In [56]:
if __name__ == "__main__":

    now = datetime.datetime.now()
    print (now)
    
    # Ruta principal donde comienza la extracción
    ppal_dir        = './ArchivosProcesar'       
    
    ## Recorre todas las carpetas y subcarpetas para extraer los archivos     
    loop_folders_files(ppal_dir)
    
    # Elimina y crea las tablas en Teradata
    create_and_drop_tables_teradata()
    
    # Inserta el contenido de los archivos a las tablas en Teradata
    insert_files_csv_in_teradata()
    
    now = datetime.datetime.now()
    print (now)

2020-05-11 18:48:13.301321
INSERTANDO ARCHIVOS A LAS TABLAS..

INSERTANDO EN EDW_TEMPUSU.LAC_LEVANTAMIENTO_TABLAS_SP
INSERTANDO EN EDW_TEMPUSU.LAC_LEVANTAMIENTO_TABLAS_VISTAS
2020-05-11 18:49:18.978045
